<img src="https://www.mlflow.org/docs/latest/_static/MLflow-logo-final-black.png" width="200">
#### MLflow Lab Part 1: Tracking Module
         
The goal of this example is to show how easy and straightforward it is to implement MLflow to a standard ML model.

**Simple steps to convert a ML model to MLflow:**
1. Import the required MLflow packages
2. Use MLflow Automatic Logging to log every metric and parameter of the model.
3. Iniciate the training model with `mlflow.start_run` to start logging.

#### Classification Problem using CNNs with Tensorflow

This is a very simple classification problem where we will be using the Fashion MNIST dataset from Keras.

<img src="https://timesofdatascience.com/wp-content/uploads/2019/02/fashion-846x515.jpg"
         alt="Fashion MNIST dataset " width="400">
         
**The Fasion MNIST dataset includes:**

* 60,000 training examples
* 10,000 testing examples
* 10 classes 
* 28x28 grayscale/single channel images

**The ten fashin labels include:**

1. T-shirt/top
2. Trouser/pants
3. Pullover shirt
4. Dress
5. Coat
6. Sandal
7. Shirt
8. Sneaker
9. Bag
10. Ankle boot

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys

import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout, GlobalMaxPooling2D
from tensorflow.keras.models import Model

#import mlflow
#from mlflow import pyfunc
#import mlflow.tensorflow

Declared various Utitily class utility methods: 
 Declared load_data( path:file_name ) for data files 
 Declared plot_graphs( ... ) for ploting artifacts 
 Declared get_mlflow_directory_path for mlruns path 
 Declared get_temporary_directory_path( ... ) temporary artifact files 
 Declared plot_residual_graphs( ... ) for plotting residues 
 Declared print_pandas_dataset( name:String ) to print Pandas dataFrame

In [0]:
print(tf.__version__)

Declared RandomForestRegress Model Class with methods: 
 Declared model() returns existing instance of Random Forest Model 
 Declared new_instance(params={...} returns a new instance of RandomForestClassifierModel 
 Declared mlflow_run(DataFrame, run_name="name") returns experiment_ID, run_ID

### Load the Dataset

In [0]:
# Load in the data
fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Increase one dimension so it can be used by the 2D convolutional keras layer
x_train = np.expand_dims(x_train,-1)
x_test = np.expand_dims(x_test,-1)
print("x_train.shape:", x_train.shape)


### Run the Model

In [0]:
def run_model(params):
  # number of classes
  K = len(set(y_train))
  print("number of classes:", K)
  # Build the model using the functional API
  i = Input(shape=x_train[0].shape)
  x = Conv2D(32, params['convSize'], strides=2, activation='relu')(i)
  x = Conv2D(64, params['convSize'], strides=2, activation='relu')(x)
  x = Conv2D(128, params['convSize'], strides=2, activation='relu')(x)
  x = Flatten()(x)
  x = Dropout(0.2)(x)
  x = Dense(512, activation='relu')(x)
  x = Dropout(0.2)(x)
  x = Dense(K, activation='softmax')(x)

  model = Model(i, x)

  # Compile and fit
  # Note: make sure you are using the GPU for this!
  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  r = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=params['epochs'])


In [0]:
for epochs, convSize in [[3,2], [15,3]]:
  params = {'epochs': epochs,
            'convSize': convSize}
  run_model(params)